# Purpose of this notebook

What if, based on just the descriptions, 
we want to see whether a project around the topic of social security ends up profiling individuals. 

There are over five hundred items to check, and clicking each of them would be slow busywork.

[The algoritmes.overheid.nl site](https://algoritmes.overheid.nl) has topic filtering (...but we're not entirely sure how complete),
and has CSV and XSLX export (...but not at the same time), so we can start handling that as text data.

But before we get to that:

### Considering that task

We described two things to address:
- Ensure that we are considering all cases we care about
- Classify as worrisome or not

In theory you can see both as classification tasks.
- social topic or not? (as a filter)
- has worrisome phrasing or not? (probably to score)


While classification is automatic,
and large enough documents have a tendency to supply good-enough indicators,
in this case there is little text to work on.

This also makes phrasing important - just hedging your language well enough would probaly make something pass.
We suspect that even _if_ there were no issue in the time cost in reading all that,
people doing that busywork would probably still have trouble, and might even disagree.

Sure, clearer cases may have some red-flag words --  _but_ some cases are just too vague even forwell-informed human estimation. 
When the information isn't there for us even with a little more real-world knowledge, it's probably less there for machines.

In particular the last case of the next list suggests a real difference between 
"lack of topical words that make us worry; probably fine", and "lack of topical words; should ask for them".

Consider some real cases:

* [Model van Bijstand naar Werk](https://algoritmes.overheid.nl/nl/algoritme/model-van-bijstand-naar-werk-gemeente-den-haag/71627856#verantwoordGebruik)
  - worrisome: 
    - the topic of 'bijstand'; terms like 'op maat', 'dossiers', 'decision tree', 'bepaalde segmenten'
    - the goal is to invite _specific_ people
  - alleviates?
    - 'eenmalig' suggests removal after suggestion
    - system _advises_ an invitation, does not decide 

* [Heronderzoeken Uitkeringsgerechtigden](https://algoritmes.overheid.nl/nl/algoritme/heronderzoeken-uitkeringsgerechtigden-gemeente-rotterdam/36585638#verantwoordGebruik)
  - worrisome: the topic of 'uitkering'; terms like 'kansberekening', 'risico-inschattingsgetal', 'historische gegevens', 'voorspellend', 'gezinssituatie'
  - alleviates: system _advises_ a review, does not decide or execte one?

* [Onderzoekswaardigheid: Slimme check levensonderhoud](https://algoritmes.overheid.nl/nl/algoritme/onderzoekswaardigheid-slimme-check-levensonderhoud-gemeente-amsterdam/95794697#verantwoordGebruik)
  - worrisome: terms like 'score', 'onderzoek', 'leeftijd', 'geboorteland'
  - alleviates:
    - system _advises_ a review, does not make decisions  ('onderzoekswaardig')
    - (explains that it addresses ethics in that it should be more equal-opportunity about scrutiny, though not exactly how much profiling is or isn't involved in that)

* [Werkverkenner](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#verantwoordGebruik)
  - worrisome:
    - topic of 'uitkering', terms like 'score', the [15-item list of personal information that it uses](https://algoritmes.overheid.nl/nl/algoritme/werkverkenner-uitvoeringsinstituut-werknemersverzekeringen/11248112#werking) 
  - alleviates:
    - said to involve checking the information that led to the invitation
    - external verification of ethics

---
Possibly okay?

* [Wmo-voorspelmodel](https://algoritmes.overheid.nl/nl/algoritme/wmovoorspelmodel-gemeente-den-haag/97246956#verantwoordGebruik)
  - worrisome: 'voorspelmodel' 
  - alleviates: results are about collective, not personal use; basis might still be personal info but 
  'open data' suggests not

* [Vroegsignalering](https://algoritmes.overheid.nl/nl/algoritme/vroegsignalering-gemeente-roosendaal/73933449#verantwoordGebruik)
  - worrisome: 'schuldeisers', 'automatisch', 'persoonsgegevens'
  - alleviates: 'advies' suggests suggestion rather that decision system; 'handmatig' suggests this is about automating, not about doing things particularly differently

* [Rechten Rotterdammers](https://algoritmes.overheid.nl/nl/algoritme/rechten-rotterdammers-gemeente-rotterdam/33569518#verantwoordGebruik)
  - worrisome: topic like uitkering, 'beslisregels', 'toekennen' based on its output
  - alleviates: 'advies', 'medewerker'+'ondersteunt'
Regelgebaseerd algoritme stelt rechten vast (geen risicoscores) 

* [Automatische kwijtschelding](https://algoritmes.overheid.nl/nl/algoritme/automatische-kwijtschelding-gemeente-rotterdam/54699221#verantwoordGebruik)
  - worrisome: 'combineren'+'informatie', 'inkomstengegevens', 'financiële' en 'huishouden' 
  - alleviates: 'opt-in'; 'handmatig'

---
Unsure?

* [DIAfragma](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#verantwoordGebruik)
  - worrisome: terms like 'integraal klantbeeld', 'BSN-nummers', 'geslacht', 'koppeling',  [list of 10+ pieces of personal information](https://algoritmes.overheid.nl/nl/algoritme/diafragma-gemeente-amsterdam/38895425#werking) 
  - alleviates: specifically avoids using some information that might lead to discrimination?

* [Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning)](https://algoritmes.overheid.nl/nl/algoritme/sociaal-domein-pko-kennissystemen-proces-kennisondersteuning-gemeente-arnhem/65732191#verantwoordGebruik)
  - worrisome: topic of uitkering, terms like 'beslisboom', 'controleert', 'door het systeem getrokken conclusies' 
  - alleviates: 'nalopen'
  - not a lot of contentful words to go on, though?




What if we limit ourself to marking how interesting cases are for us to potentially _look at_,
rather that decide _how bad they are_.

We only support a human in making their decisions easier
(the same thing we want to see in some of these algoritms themselves),

To scrape as much as we can from minimal text, 
we might look at worrisome words, which verbs appear in the same sentence? Paragraph?

There sould probably be some suggestion of phrases to include,
based on being related to what you have already decided is good or bad,
or just on being contentful words not currently under consideration.



That said, this approach also comes with different fundamental limitations. 

For example, even smarter NLP might miss negations (even recent developments, LLMs, used to be infamously bad at seeing negations).
Consider detecting 'geautomatiseerde besluitvorming' and miss that the words before were 'Er is geen sprake van';
you might see 'BSN-nummers' and 'koppeling' but mis the words 'zonder verdere' inbetween.
You might see 'handmatig' but not tell whether it is a thing we are removing, or ensuring is still there.

Things like "Risico's bij het gebruik van het algoritme zijn mede vanwege de mogelijkheid om in te grijpen op de door het systeem getrokken conclusies niet noemenswaardig, waarmee de proportionaliteit in orde is." are a feat of sentence nesting, and we would like an extra connecting argument that actually leads to that final "it's probably fine".


But then, if we're not even looking for interpretation like negatives, we are _certain_ to miss those differences.
That said, it is not necessarily a bad thing to bring up these cases, if only because some cases lie fully in context.

"Het algoritme kan alleen automatisch een aanvraag/aangifte goedkeuren. Een aanvraag/aangifte afkeuren kan alleen de ambtenaar." may sound like good bias, or bad,
depending on what it is we're okaying.

## The data

### Fetching data

In [1]:
import csv, io, random, pprint
import wetsuite.helpers.net
import wetsuite.helpers.strings
import wetsuite.helpers.spacy
import wetsuite.helpers.lazy

In [2]:
# higher timeout because it takes take 10 to 20 seconds to produce that
csv_bytes = wetsuite.helpers.net.download( 'https://algoritmes.overheid.nl/api/downloads/NLD?filetype=csv', timeout=60 )  

# some manual decoding because it seems to has UTF-8 with a BOM which is not invalid but is sort of pointless, and not something a lot of things look for / look to remove.
csv_str = csv_bytes.decode('utf-8').lstrip('\ufeff') # Remove that, move on.
# Now we have CSV in a (unicode) string.

In [3]:
# CSV is not a single standard, it has flavours that we need to get the right one of.
# This seems good for to this data:
class AlgoritmRegisterCSVDialect(csv.Dialect):
    header         = True
    lineterminator = '\n'
    delimiter      = ','
    doublequote    = True
    quotechar      = '"'
    quoting        = csv.QUOTE_ALL

csv_parsed = csv.DictReader( io.StringIO(csv_str) , dialect=AlgoritmRegisterCSVDialect())
# csv.DictReader seems to generate data as it goes
# ...so gives the data once - lets put that into a list so we can examine it repeatedly
algodict_list = list( csv_parsed ) # a list of dicts, where each dict shares the same keys:

### Inspecting data

In [4]:
print( 'Number of algorithm:', len(algodict_list) )

print( 'Columns/key names:', list( algodict_list[0].keys() ) )

Number of algorithm: 513
Columns/key names: ['name', 'organization', 'description_short', 'category', 'website', 'status', 'goal', 'proportionality', 'lawful_basis', 'standard_version', 'url', 'contact_email', 'lang', 'publiccode', 'source_data', 'methods_and_models', 'human_intervention', 'risks', 'provider', 'process_index_url', 'tags', 'source_id', 'begin_date', 'end_date', 'impacttoetsen', 'publication_category', 'lawful_basis_grouping', 'impacttoetsen_grouping', 'source_data_grouping', 'algorithm_id', 'type', 'iama_description', 'uuid', 'lawful_basis_link', 'source_data_link', 'department', 'impact', 'decision_making_process', 'documentation', 'competent_authority', 'iama', 'dpia', 'dpia_description', 'objection_procedure', 'area', 'revision_date', 'description', 'application_url', 'mprd', 'monitoring', 'performance_standard']


In [5]:
import pandas # pandas is a handy tool to putting that into a table and
with pandas.option_context('display.min_rows', 5): # temporarily restrict to show just a few rows, we are mostly pointing out the columns, not the cases 
    df = pandas.DataFrame(algodict_list) # pandas deals with lists of dicts by making keys columns -- so in this case it needs no further instruction to make it a table:
    display(df)

,name,organization,description_short,category,website,status,goal,proportionality,lawful_basis,standard_version,...,dpia,dpia_description,objection_procedure,area,revision_date,description,application_url,mprd,monitoring,performance_standard
0,Dashboard evaluatie cameratoezicht,Gemeente Amsterdam,Data-analyse voor de evaluatie van de noodzaak...,Openbare orde en veiligheid,Beleidsregel cameratoezicht ter handhaving van...,In gebruik,Cameratoezicht kan ingesteld worden om de open...,Burgemeesters kunnen voor de handhaving van de...,,1.0,...,,,,,,,,,,
1,Leefomgeving: Legesberekening,Gemeente Westerkwartier,Om leges te kunnen heffen moeten klanten grond...,,,In gebruik,Leges zijn feitelijk vergoedingen voor gemeent...,De grondslagen en de berekening moeten in over...,Elke gemeente stelt jaarlijks de zogenaamde le...,1.0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,Telefonisch Innen (Wet Wahv),Centraal Justitieel Incassobureau,Eén van de belangrijkste maatschappelijke opga...,Wahv,https://www.cjib.nl/telefonisch-innen,In gebruik,,,,0.1,...,,,,,,,,,,
512,Vergunningverlening; Wonen,Gemeente Utrecht,Het uitvoeren van de algemene leefbaarheidstoe...,,,,,,,0.1,...,,,,,,,,,,


In [6]:
# inspect some fields
display( df.value_counts('status') )

display( df.value_counts('publication_category') )

catcounts = df.value_counts('category')
display( catcounts[catcounts>1] )

status
In gebruik         435
In ontwikkeling     45
Buiten gebruik      32
                     1
Name: count, dtype: int64

publication_category
Impactvolle algoritmes    240
Overige algoritmes        231
Hoog-risico AI-systeem     24
                           18
Name: count, dtype: int64

category
Organisatie en bedrijfsvoering                                             168
Sociale zekerheid                                                           57
Openbare orde en veiligheid                                                 39
                                                                            37
Verkeer                                                                     26
Ruimte en infrastructuur                                                    25
Overheidsfinanciën                                                          15
Natuur en milieu                                                            13
Wonen                                                                        9
Zorg en gezondheid                                                           8
Economie, Ruimte en infrastructuur                                           8
Economie, Ruimte en infrastructuur, Verkeer                                  6
Recht                                      

## Working our way to do that task

### Selecting just the interesting text fields

In [7]:
# starting from the list of keys above, remove things we do not find interesting.
#   so this is a subset (also a slight reordering, )
#   the commented-out things are still _potentially_ useful - play with it to see
our_key_choices = [ 
        'algorithm_id',           # would let us link to the site again
        #'publication_category',
        'name', 
        'organization',
        'category',
        'description_short',
        'goal',
        'proportionality',
        'source_data',
        'methods_and_models',
        'human_intervention',
        'risks',
        'tags',
        #'status',
        #'begin_date', 'end_date',
        #'impacttoetsen',
        #'impacttoetsen_grouping',
        #'source_data_grouping', 
]

# do the selection as a pandas table, so we can get a quick overview of what we made
smaller_table = df[our_key_choices]
display(smaller_table)

,algorithm_id,name,organization,category,description_short,goal,proportionality,source_data,methods_and_models,human_intervention,risks,tags
0,58953176,Dashboard evaluatie cameratoezicht,Gemeente Amsterdam,Openbare orde en veiligheid,Data-analyse voor de evaluatie van de noodzaak...,Cameratoezicht kan ingesteld worden om de open...,Burgemeesters kunnen voor de handhaving van de...,Alleen de politie heeft vrij toegang tot de se...,De registraties die de politie maakt worden vi...,De weergave van de registraties geeft de belei...,"We gebruiken slechts noodzakelijke gegevens, d...","Cameratoezicht, dashboard, handhaving openbare..."
1,26378451,Leefomgeving: Legesberekening,Gemeente Westerkwartier,,Om leges te kunnen heffen moeten klanten grond...,Leges zijn feitelijk vergoedingen voor gemeent...,De grondslagen en de berekening moeten in over...,,Vast tarief: Bij deze berekeningsmethode wordt...,Verantwoordelijke en gemachtigde medewerkers a...,Omdat gewerkt wordt met persoonsgegevens voor ...,
2,75856898,Veilig Alternatief/Top 600,Gemeente Amsterdam,Openbare orde en veiligheid,Om de stad veiliger te maken coördineert de ge...,Om de stad veiliger te maken coördineert de ge...,,Personen kunnen alleen instromen in de Top600 ...,Elke binnenkomende dataset is door de aanlever...,De samenstelling van de lijsten wordt gedaan d...,De methodiek om personen te selecteren voor de...,Veilig AlternatiefTop 600Actiecentrum Veilighe...
3,51149143,iBurgerzaken e-diensten,Gemeente Westerkwartier,Organisatie en bedrijfsvoering,Inwoners en ondernemers van Nederlandse gemeen...,Het doel van de e-diensten met het onderliggen...,Alle aanvragen via de balie afhandelen is voor...,"Gegevens die worden gebruikt komen uit de BRP,...",Via de website van de gemeente kan een inwoner...,Het algoritme gaat uit van een positief scenar...,Het is aan gemeenten om vorm te geven aan het ...,
4,37694526,Dashboard overzicht demonstraties,Gemeente Amsterdam,Openbare orde en veiligheid,Data analyse ten behoeve van voorbereidingen e...,Het doel van het dashboard demonstraties is he...,We krijgen in Amsterdam jaarlijks veel aanmeld...,De aangemelde demonstraties komen als Kennisge...,Architectuur van het model:Het systeem geeft e...,Met de datavisualisatie krijgen de beleidsmede...,Demonstreren is een grondrecht. Het staat mens...,"Demonstraties, Dashboard, openbare orde en vei..."
...,...,...,...,...,...,...,...,...,...,...,...,...
508,23448142,Sancties,Sociale Verzekeringsbank,,Een beslistool helpt onze medewerkers om te be...,Algoritmes helpen bij moeilijke beslissingen w...,,Onze belangrijkste bronnen zijn uw gegevens in...,Beslisboom (rule-based),,,
509,96585577,Vrijwillige verzekering AOW en Anw,Sociale Verzekeringsbank,,Een algoritme berekent de hoogte van de premie...,Algoritmes helpen bij het nuttig uitvoeren van...,,Onze belangrijkste bronnen zijn:\r\n\r\npersoo...,Beslisboom (rule-based),,,
510,28915384,Preventie & Handhaving,Sociale Verzekeringsbank,,Met dit model voorspellen wij hoe groot de kan...,Door het SWAN-model kunnen wij ons richten op ...,,Onze belangrijkste bronnen zijn interne gegeve...,Zelflerend,,Wij toetsen onze modellen altijd of ze voldoen...,
511,41914566,Telefonisch Innen (Wet Wahv),Centraal Justitieel Incassobureau,Wahv,Eén van de belangrijkste maatschappelijke opga...,,,,,,,


In [8]:
# Say we want to flatten/merge all that text to one string
# The column selection part is easy to do in pandas (we just did),
#   but addressing rows in pandas is somewhat awkward,
#   so we go back to the earlier data.  Same key names, after all.

ourdata = {}
for item_dict in algodict_list:
    algo_id = item_dict.get('algorithm_id')
    text_fragments = [] 
    for text_field_name in our_key_choices[1:]: # skip the first (algorithm_id)
        text_fragments.append( item_dict.get(text_field_name) )
    ourdata[algo_id] = { # we may not use all these fields, but it's handy
        'algo_id':algo_id, 
        'textlist':text_fragments, 
        'titel':text_fragments[0], 
        'category':item_dict.get('category'),
        'sociaal':set(),
        'worrying':set(),
        'alleviates':set(),
    }


#display( random.choice( list(ourdata.items()) ) ) # show a random one of them, as an example

### Noting the 'enthusiastic dutch compounding makes for out-of-vocabulary words' problem

NLP likes to see words as 'character sequences without spaces' because that's a lot easier to tokenize into.

Dutch likes smashing together words any chance it gets, which means it's better than other languages
at making words that aren't very common in training data, known at all, and/or are hard to estimate meaning of.

We normally might not care so much, because for larger documents we are likely to catch the topic 
even if we miss a few words, but here we might need every word we can get. 

A thing to keep in mind. We may use libaries like spacy sparingly/informedly.
<!--
#     # for nc in doc.noun_chunks:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( nc.text ) )
#     # for ent in doc.ents:
#     #     ph.add( wetsuite.helpers.strings.remove_deheteen( ent.text ) )
#     # if len(ph)>0:
#     #     data[id]['ph'].extend( ph )

#     # data[id]['ph'] = ( wetsuite.helpers.strings.ordered_unique(data[id]['ph'], case_sensitive=False) )

# pprint.pprint( data )
# -->

In [10]:
random_case_dict = random.choice( list(ourdata.values()) )
random_case_text = ' '.join( random_case_dict['textlist'] )

doc = wetsuite.helpers.lazy.spacy_parse( random_case_text )
display( wetsuite.helpers.spacy.notebook_content_visualisation(doc) )

## Filtering for sociale zekerheid

Assuming we don't trust the field/filter the site gives us, 
or at least want to check it leaves out nothing, let's try our own, and compare.

Right now these are used as substrings, so will match even if part of larger words (which is why inflections are removed, or should be),
done intentionally to alleviate the compounding - e.g. bijstand will match bijstandnorm, bijstandgerectigt

This is not very obvious in the later table, and probably should be (e.g. 'actually report the whole thing we matched'), 
but it would take some more code to make this easy to use. 

In [11]:
# substrings that indicate this is a sociale
social_indicators = '''bijstand
uitkering
werkloosheidswet
IOAZ
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen 
IOAW
Wet inkomensvoorziening oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers
IOW
Wet inkomensvoorziening oudere werklozen
de Wmo
Wet maatschappelijke ondersteuning
WWB
wet werk en bijstand
Wet LB
Wet op de loonbelasting
Awir
Algemene wet inkomensafhankelijke regelingen
Ziektewet
Participatiewet
UWV
AOW
arbeidsongeschikt
arbeidsbeperking
arbeidsvermogen
WAJONG
schuldhulpverlening
loonwaarde 
levensonderhoud
re-integratie
reïntegratie
werkhervatting
Hulp bij het Huishouden
Ondersteuning thuis 
betalingsachterstand
Kinderbijslag
kinderopvang
toeslagpartner
Werk en Inkomen
ziekmelding
Rechtsbijstand 
kwijtscheld
zorgaanbieder
ziekengeld
woonlandbeginsel
alimentatie
Sociale Verzekerings
Sociale zekerheid'''.lower().split('\n') # ww   sociale

# substrings that make us pay attention. Again: this is not very robust
worrisome_indicators = '''voorspel
beslis
burgerservicenummer
BSN
Sofinummer
Wet basisregistratie personen
BRP
profilering
individu
risico
score
gestuurd
onderzoek
kansberekening
inschattingsgetal
dossier
uw gegevens
combineren
woonadres
inkomstengegevens
vermogensgegevens
voertuiggegevens
financiele gegevens
financiële gegevens
financiele informatie
financiële informatie
gezinssituatie
gezinsituatie
leeftijd
functie
huishouden
Geboortedatum
persoonsgegevens
naamgegevens
geboortegegevens 
ziekmelding
leeftijd
geboorteland
nationaliteit
decision tree
beslisboom
rule-based
op maat
bijstand
uitkeringsgerechtigden
werkzoekend
zonder tussenkomst van
historische gegevens
schuldeiser
schuldenaren
schuldenaar
toeslagen
heronderzoeken
datamodel'''.lower().split('\n') # heronderzoek


alleviating_indicators = '''handmatig
handmatig bekijken
handmatig controleren
niet leidend
advies
eenmalig
ondersteun
foutgevoelig
Algemene Verordening Gegevensbescherming
AVG
profilering
mogelijke discriminatie
kwijtschelding
Niet zelflerend
privacy
privacy impact 
privacy impact assessment'''.lower().split('\n')

In [12]:
import textwrap, re, math
filtered_data = ourdata # all
filtered_data = {algoid:details  for algoid, details in ourdata.items()  if 'sociale zekerheid' in details.get('category').lower()} # just the ones already tagged this way

for algo_id, details in filtered_data.items():
    category = details.get('category')
    textlist = details.get('textlist')
    text = '  '.join(textlist)

    for szi in social_indicators:
        if re.search(r'\b'+szi, text.lower()) is not None:
            details['sociaal'].add( szi )
    for wi  in worrisome_indicators:
        if re.search(r'\b'+wi, text.lower()) is not None:
            details['worrying'].add( wi )
    for ai  in alleviating_indicators:
        if re.search(r'\b'+ai, text.lower()) is not None:
            details['alleviates'].add( ai )

    lds, lw, la = len(details['sociaal']), len(details['worrying']), len(details['alleviates'])
    # WARNING: DUMB SCORE -- to calculate anything even resembling a real store would take a lot more than this
    score = round( 
        math.log( 1+ len(details['sociaal']))  +  math.log(1+len(details['worrying']))  -  len(details['alleviates'] ), 
        1)
    details['dumb_review_score'] = round(score,1)

with pandas.option_context('display.min_rows', 600, 'display.max_rows', 600):

    df = pandas.DataFrame(filtered_data.values()) # make it a pandas table again
    #display(df)
    display(
        df[:]  .drop('textlist', axis=1)    # make copy just for display, select all but one column from it
        .sort_values('dumb_review_score',ascending=False)
        .style.format({
            'algo_id': lambda x:'<a href="https://algoritmes.overheid.nl/nl/algoritme/IGNOREME/%s#algemeneInformatie">%s</a>'%(x,x), #  make algorithm IDs link to the website
            'sociaal': lambda x:', '.join(x),    # (the rest of these are minor formatting we could do without)
            'worrying': lambda x:', '.join(x),
            'alleviates': lambda x:', '.join(x),
            'dumb_review_score': lambda x:'%.1f'%x
        }) 
    )

,algo_id,titel,category,sociaal,worrying,alleviates,dumb_review_score
41,86726997,Toekenning Bijverdienbeloning,"Sociale zekerheid, Werk, Overheidsfinanciën","ioaw, uitkering, sociale zekerheid, participatiewet, ioaz, bijstand","risico, dossier, financiele gegevens, toeslagen, bijstand",,3.7
34,19118228,Sociaal Domein: eDiensten voor aanvragen,Sociale zekerheid,"uitkering, sociale zekerheid, bijstand","risico, burgerservicenummer, brp, gestuurd, beslisboom, beslis, bsn, bijstand",,3.6
23,52368282,Sociaal domein: eDienst aanvragen levensonderhoud,Sociale zekerheid,"sociale zekerheid, levensonderhoud, bijstand","risico, financiële gegevens, brp, beslisboom, beslis, bijstand",,3.3
17,92734630,Uitkering Participatiewet,Sociale zekerheid,"participatiewet, uitkering, werk en inkomen, sociale zekerheid","beslis, risico, persoonsgegevens, zonder tussenkomst van",,3.2
16,26932183,Re-integratie Werk arrangement,Sociale zekerheid,"participatiewet, re-integratie, sociale zekerheid","beslis, risico, persoonsgegevens, zonder tussenkomst van",,3.0
8,29323438,Berekening van alimentatie,Sociale zekerheid,"sociale zekerheid, alimentatie, levensonderhoud, bijstand","risico, leeftijd, bijstand",,3.0
10,36585638,Heronderzoeken Uitkeringsgerechtigden,Sociale zekerheid,"re-integratie, uitkering, sociale zekerheid","onderzoek, risico, uw gegevens, historische gegevens, heronderzoeken, leeftijd, nationaliteit, voorspel, uitkeringsgerechtigden, individu, gezinssituatie, inschattingsgetal, werkzoekend",ondersteun,3.0
15,21277971,Re-integratie Werk Matchen,Sociale zekerheid,"participatiewet, re-integratie, sociale zekerheid","onderzoek, risico, persoonsgegevens, zonder tussenkomst van",,3.0
63,94414194,Vaststellingsmodel regres,Sociale zekerheid,"uitkering, wajong, sociale zekerheid, uwv, arbeidsongeschikt, ziekmelding","onderzoek, risico, burgerservicenummer, bsn, persoonsgegevens, ziekmelding",privacy,2.9
11,65732191,Sociaal Domein: PKO Kennissystemen (Proces & Kennisondersteuning),"Sociale zekerheid, Organisatie en bedrijfsvoering","uitkering, werk en inkomen, sociale zekerheid, bijstand","risico, heronderzoeken, brp, beslisboom, individu, beslis, bijstand",ondersteun,2.7
